In [1]:
from pytorch_libraries.model_NeuralStyleTransfer import *
from IPython.display import clear_output
from pylab import *
%matplotlib inline

In [2]:
# Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([transforms.Resize(250),
                                transforms.ToTensor()])
style   = Variable(load_image('../data/images/s3.jpg', transform))
content = Variable(load_image('../data/images/00.jpg', transform))
target  = Variable(content.data.clone(), requires_grad=True)

vgg     = VGGNet()#.cuda()

In [3]:
optimizer = torch.optim.Adam([target], lr=5e-3)
    
STYLE_WEIGHT = 1e5
TOTAL_OPTSTEP= 300

In [4]:
 for step in range(TOTAL_OPTSTEP):
        
        # Extract multiple(5) conv feature vectors
        target_features  = vgg(target)
        content_features = vgg(content)
        style_features   = vgg(style)

        style_loss = 0
        content_loss = 0
        
        decay = 1
        # for each layer
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            
            # Compute content loss (target and content image)
            content_loss += torch.mean((f1 - f2)**2) * decay
            decay *= 0.1

            # Reshape conv features
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)
            # Compute gram matrix  
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())
            # Compute style loss (target and style image)
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) 

        # Compute total loss, backprop and optimize
        loss = content_loss + STYLE_WEIGHT * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % 10 == 0:
            print ('Step [%d/%d], Content Loss: %.4f, Style Loss: %.4f' 
                   %(step+1, TOTAL_OPTSTEP, content_loss.data[0], style_loss.data[0]))
    
        if (step+1) % 10 == 0:
            # Save the generated image
            #denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().cpu().squeeze()
            img = img.data.clamp_(0, 1)
            torchvision.utils.save_image(img, 'output-%d.png' %(step+1))

Step [10/300], Content Loss: 0.0309, Style Loss: 2.4085
Step [20/300], Content Loss: 0.0513, Style Loss: 1.2712
Step [30/300], Content Loss: 0.0585, Style Loss: 0.8615
Step [40/300], Content Loss: 0.0616, Style Loss: 0.6306
Step [50/300], Content Loss: 0.0635, Style Loss: 0.4933
Step [60/300], Content Loss: 0.0651, Style Loss: 0.4039
Step [70/300], Content Loss: 0.0665, Style Loss: 0.3410
Step [80/300], Content Loss: 0.0679, Style Loss: 0.2945
Step [90/300], Content Loss: 0.0691, Style Loss: 0.2585
Step [100/300], Content Loss: 0.0702, Style Loss: 0.2299
Step [110/300], Content Loss: 0.0712, Style Loss: 0.2068
Step [120/300], Content Loss: 0.0722, Style Loss: 0.1878
Step [130/300], Content Loss: 0.0731, Style Loss: 0.1718
Step [140/300], Content Loss: 0.0739, Style Loss: 0.1582
Step [150/300], Content Loss: 0.0746, Style Loss: 0.1465
Step [160/300], Content Loss: 0.0753, Style Loss: 0.1364
Step [170/300], Content Loss: 0.0760, Style Loss: 0.1275
Step [180/300], Content Loss: 0.0766, St